
|  |
| ------------------------------------------------------- | 
| ![Tremplin des sciences](images/tremplinColorSmall.png) | 

Cahier d'exercices pour l'enseignement et l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Le dispositif clef en main repose sur l'utilisation d'une RaspberryPi chargée avec le système d'exploitation Debian enrichi, fourni par le projet. Les sources et les exécutables sont accessibles dans [l'espace collaboratif de la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/) systèmes d'exploitation sur [la page des OS  de Raspberries Pi](http://mediaserv.climatetmeteo.fr/images/RaspBerry/DebianStretchPi3/).  Toutes les ressources issues du projet sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ou sous les licences libres d'origine des outils utilisés. Les ressources  du projet peuvent être utilisées dans tout autre environnement compatible.![licence : Creative Commons](images/Licence.jpg) 

Auteurs : G. Vidal, C-H. Eyraud, E. le Jan

------------------------------------------------------------

# Le montage

## Utilisation d'un capteur analogique

Certains capteurs en particulier le capteur de niveau d'eau ou capteur d'humidite du sol  fournissent un signal analogique; c'est pourquoi il faut les convertir en signal numérique pour pouvoir les exploiter avec une RaspberryPI

![Capteur de niveau d'eau ](images/WaterSensor.jpg)  |  ![Capteur d'humidité](images/MoistureSensor.jpg)


## Le convertisseur Analogique --> digital

Nous avons à notre disposition deux convertisseurs analogique numérique dont le fonctionnement est identique. Ce qui les diffférencie est la grandeur privilégiée: fréquence d'échantillonnage pour le premier, précision de l'échantillonnage pour le second.

 * [ADS1015](https://iotdk.intel.com/docs/master/upm/classupm_1_1_a_d_s1015.html) Résolution de 12 bit, Fréquence d'échantillonnage Min : 128 Hz Max :3300 Hz (SPS_128 à SPS_3300)
 * [ADS1115](https://iotdk.intel.com/docs/master/upm/classupm_1_1_a_d_s1115.html) Résolution  de 16 bit, Fréquence d'échantillonage Min : 8 Hz Max : 860 Hz (SPS_8 à SPS_860)

Ce qui  diffférencie les deux convertisseurs est la grandeur privilégiée:
* pour ADS1015 c'est la fréquence de l'échantillonnage 
* pour ADS1115 c'est la précision  de l'échantillonage.

La totalité des commandes sont communes sauf le réglage de la fréquence d'échantillonage, dans les hautes fréquences pour le `ADS1015`et dans les basses fréquences pour le `ADS1115`.

Deux modes de fonctionnement sont possibles :
 * en mode différentiel le signal est mesuré par différence entre la broche de sortie et la masse
 * en mode direct le signal est mesuré directement sur la broche de sortie
 
## Le convertisseur de niveau logique (level-shifter)
 
 Le capteur étant alimenté en 5V, il faut abaisser la tension du signal de retour pour que la broche du Raspberry ne reçoive qu'une tension de 3,3V. C'est le rôle du composant bidirectionnel Adafruit [TXB014](https://www.adafruit.com/product/1875) qui permet ici d'abaisser la tension sur le bus I2C
 
 ![Capteur de niveau d'eau ](images/LogicLevelConverter.jpg) | à gauche la basse tension (LV Low Voltage), à doite la haute tension (High Voltage)

La figure ci-dessous présente l'ensemble du dispositif :

![Convertisseurs vue générale](images/CapteurNiveauDEau.jpg)
                        ![licence : Gérard Vidal](images/Licence.jpg)
                        
Le câblage du convertisseur ne présente pas de difficulté particulière, le point requérant un peu d'attention est le choix de l'adresse qui sera envoyée sur le bus `I2C` car il faut choisir cette adresse parmi les 4 possibles :  `0x49` `0x48` `0x4B` `0x4A`. Le choix se fait en établissant une connexion entre la broche `ADDR` et l'une des 4 broches `VDD` `GND` `SCL` `SDA`. Le tableau ci-dessous indique les équivalences des connexions :

| Broche | Adresse |
|:------ | ------- |
| `VDD`  | `0x49`  |
| `GND`  | `0x48`  |
| `SCL`  | `0x4B`  |
| `SDA`  | `0x4A`  |

Le capteur analogique doit être alimenté en 5 Volts (cables rouges) et les retours des données sur le RaspberryPi en 3,3V (Cables oranges). Le cable bleu joignant les broches ADDR et VDD du convertisseur montre qu'il faudra prendre l'adresse `0x49` sur le bus I2C.

# Le programme
## Importation des librairies nécessaires à l'utilisation du convertisseur.
On n'importe pas de librairies propre au capteur car il s'agit d'un capteur `analogiques` qui transmet une variation de potentiel sur un fil série.

In [3]:
import mraa, upm
from time import sleep
from upm import pyupm_ads1x15 as upm

## Récupération des paramètres d'état de l'échantillonneur

On déclare ici l'échantillonneur et on récupère les valeurs de ses paramètres d'opération (gain, fréquence et mode). 
 * getGain()
 * getSPS()
 * getContinuous()
 
On fixe ensuite ces paramètres pour la manipulation que l'on souhaite effectuer.
 * setGain()
 * setSPS()
 * setContinuous()
 
Le paramètre SPS contrôle la fréquence d'échantillonnage.
* Pour ADS1015 les valeurs possibles sont : SPS_128, SPS_250, SPS_490, SPS_920, SPS_1600, SPS_2400, SPS_3300.

* Pour ADS1115 les valeurs possibles étant :  SPS_8, SPS_16, SPS_32, SPS_64,
  SPS_128, SPS_250, SPS_475, SPS_860. 
  
Le choix du gain est aussi un paramètre crucial. La valeur doit être choisie en fonction des propriétés du capteur :  `GAIN_TWOTHIRDS` = ADS1X15_PGA_6_144V, `GAIN_ONE` = ADS1X15_PGA_4_096V, `GAIN_TWO` = ADS1X15_PGA_2_048V, `GAIN_FOUR` = ADS1X15_PGA_1_024V, `GAIN_EIGHT` = ADS1X15_PGA_0_512V, `GAIN_SIXTEEN` = ADS1X15_PGA_0_256V. 

In [4]:
humidite = upm.ADS1015(0, 0x49)
gain = humidite.getGain()
rate = humidite.SPS_920
sps = humidite.getSPS()
cont = humidite.getContinuous() 
print ("Gain : {0:5d}".format(gain)," SPS : {0:5d}".format(sps), 
       "Continuous : {0:5d}".format(cont))
humidite.setContinuous(True) # Pour ne pas faire de conversion en continu
humidite.setGain(upm.ADS1X15.GAIN_ONE) # Passage à un gain 1024
humidite.setSPS(rate) # Passage à une fréquence de 128 définie par la valeur de rate précédemment : SPS_128
# Vérification
gain = humidite.getGain()
sps = humidite.getSPS()
cont = humidite.getContinuous() 
print ("\nNouvelles valeurs :")
print ("Gain : {0:5d}".format(gain)," SPS : {0:5d}".format(sps), 
       "Continuous : {0:5d}".format(cont))

Gain :   512  SPS :   128 Continuous :     0

Nouvelles valeurs :
Gain :   512  SPS :    96 Continuous :     1


## Mesure directe de la variable humidité

Comme les données du capteur sont envoyées sur la broche analogique A3 du convertisseur, la mesure se fait par la commande suivante:

In [12]:
# qual = humidite.getSample(upm.ADS1X15.SINGLE_0)
mesurehumidite = humidite.getSample(upm.ADS1X15.SINGLE_1)
# On peut aussi faire une mesureplus précise en connectant une des entrée libre du convertisseur à la masse, par exemple ici A0
# la mesure différentielle se fera alors entre A0 et A1
# mesurehumidite_diff = humidite.getSample(upm.ADS1X15.DIFF_0_1)
print ("Mesure de l'humidité : {0:.2f}\n".format(mesurehumidite))
#print ("Mesure de l'humidité : {0:.2f}\n".format(mesurehumidite), "Mesure Humidité Différentielle : {0:.2f}\n".format(mesurehumidite_diff))

Mesure de l'humidité : 2.34



# Série de mesures à l'aide d'une boucle
Dans le groupe suivant on répète 5 fois la mesure en partant de 0 et en allant jusqu'à 19 (inférieur à 20) au pas de 1 par défaut.

In [13]:
for num in range(0,3):
    mesurehumidite = humidite.getSample(upm.ADS1X15.SINGLE_1)
    print (num , "Mesure de l'humidité : {0:.2f}\n".format(mesurehumidite))    
    sleep(2)

0 Mesure de l'humidité : 2.39

1 Mesure de l'humidité : 2.37

2 Mesure de l'humidité : 2.40



# Afficher la mesure

Nous utiliserons l'[Afficheur couleur Grove](http://wiki.seeedstudio.com/Grove-LCD_RGB_Backlight/) qui sera sur le bus I2C
Voir le cahier numéro 05 qui explique les lignes de code suivantes

In [11]:
# Importation de la bibliothèque de l'afficheur
from upm import pyupm_jhd1313m1 as jdm

# déclaration des deux adresses de communication où sont envoyées les informations(Texte et couleur)
lcdAddress = 0x3E
rgbAddress = 0x62
bus1 = 0

lcd = jdm.Jhd1313m1(bus1,lcdAddress,rgbAddress)

status = lcd.clear()
status = lcd.backlightOff()

#positionnement de l'affichage
status = lcd.backlightOff()
status = lcd.clear()
lcd.setColor(255,0,0)
lcd.setCursor(1, 0)
#affichage du texte et de la valeur
status = lcd.backlightOn()
status = lcd.write("{0:.2f}".format(mesurehumidite))
sleep(2)


# effacement et extinction
status = lcd.clear()
status = lcd.backlightOff()